In [2]:
import pandas as pd
import requests
from io import StringIO
import mysql.connector
import pymysql
from sqlalchemy import create_engine
from bs4 import BeautifulSoup

### Problem 1: You are given a SQL file link: https://drive.google.com/file/d/1WFt7B84LTHhMueoKmz8W-PRo7xXqmZf3/view?usp=share_link. Read the data by using the file and store it in a excel file. In this data, there are 3 tables named "invoices", "order_leads" and "sales_sql". So create 3 sheets to your excel file.

In [18]:
# importing from database world on local server
import warnings
warnings.filterwarnings("ignore")
conn = mysql.connector.connect(host="localhost",user="root",password="",database="world")
df1 = pd.read_sql_query("select * from city",conn)
df2 = pd.read_sql_query("select * from country",conn)
df3 = pd.read_sql_query("select * from countrylanguage",conn)

In [21]:
#exporting data into excel file in different sheets
with pd.ExcelWriter("World.xlsx") as writer:
    df1.to_excel(writer,sheet_name="city",index=False)
    df2.to_excel(writer,sheet_name="country",index=False)
    df3.to_excel(writer,sheet_name="countrylanguage",index=False)

### Problem 2: Go to the site: https://rapidapi.com/wirefreethought/api/geodb-cities. From here, you have to grab the API and have to choose proper routes to get the cities of different countries. After getting the right API, hit that API and create a dataframe of all the cities that you can get by using the API. Then store the dataframe to a SQL. If you need to create an account or have to subscribe, then do that (it has free subscription but has some limitations. Use that free subscription and modify your accordingly to get all the data).

In [88]:
url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities"
headers = {
	"x-rapidapi-key": "439f436758mshe81e47711e76d8ap1e2ed2jsnc25cfce27a46",
	"x-rapidapi-host": "wft-geo-db.p.rapidapi.com"
}
response = requests.get(url,headers=headers)
data = response.json()["data"]
next_url = response.json()["links"][1]["href"]
count = response.json()["metadata"]["totalCount"]
pd.DataFrame(data)

,id,wikiDataId,type,city,name,country,countryCode,region,regionCode,regionWdId,latitude,longitude,population
0,3350606,Q24668,CITY,Aixirivall,Aixirivall,Andorra,AD,Sant Julià de Lòria,06,Q24282,42.462450,1.502090,1025
1,3216144,Q24656,CITY,Aixovall,Aixovall,Andorra,AD,Sant Julià de Lòria,06,Q24282,42.476358,1.489492,69
2,3406038,Q4699394,CITY,Aixàs,Aixàs,Andorra,AD,Sant Julià de Lòria,06,Q24282,42.486389,1.467222,0
3,3360277,Q24475,CITY,Ansalonga,Ansalonga,Andorra,AD,Ordino,05,Q24272,42.568443,1.521571,0
4,3341362,Q24551,CITY,Anyós,Anyós,Andorra,AD,La Massana,04,Q24276,42.534592,1.541650,1006


In [89]:
import time
n = 1
for i in range(1,count):
    n+=1
    headers = {
	"x-rapidapi-key": "439f436758mshe81e47711e76d8ap1e2ed2jsnc25cfce27a46",
	"x-rapidapi-host": "wft-geo-db.p.rapidapi.com"
    }
    url = "https://wft-geo-db.p.rapidapi.com"
    next_page = url + next_url
    response = requests.get(next_page,headers=headers)
    links = response.json()["links"]
    for i in links:
        if i["rel"] == "next":
            next_url = i["href"]
    time.sleep(2)
    data.extend(response.json()["data"])
    if n == 10:
        break


In [92]:
df = pd.DataFrame(data)
df.duplicated().sum()

0

In [95]:
engine = create_engine("mysql+pymysql://root:@localhost/test")

In [96]:
df.to_sql("cities",con=engine,if_exists="append")

50

### Problem 3: Go to this url: https://www.flipkart.com/search?q=smartphones. This is the url to find phones in flipkart website. You have to extract the below things:

- image url of the phone
- name of the image
- average ratings
- total ratings
- total reviews
- discounted price
- actual price
### Extract all the phones which are available in this website. So you have to use the pagination concept. Also after requesting every page through the url, please wait for a while (minimum 2-3 seconds), otherwise your IP address can be banned to access the flipkart website later.
### After collecting all the data, save that in a JSON file.

In [26]:
from fake_useragent import UserAgent
import time

headers = {"User-Agent":UserAgent().random}
url = "https://www.meesho.com/men-watches/pl/3k7"
try:
    response = requests.get(url, headers=headers, timeout=30)# Set 10-second timeout
except requests.exceptions.RequestException as e:
    pass

In [28]:
import requests
import random

# List of different user-agents to mimic real browsers
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
]

headers = {
    "User-Agent": random.choice(USER_AGENTS),  
    "Referer": "https://www.google.com/",  
    "Accept-Language": "en-US,en;q=0.9",
}

url = "https://www.meesho.com/men-watches/pl/3k7"

response = requests.get(url, headers=headers)

print(response.text)  # Check if content is loading


<HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;www&#46;meesho&#46;com&#47;men&#45;watches&#47;pl&#47;3k7" on this server.<P>
Reference&#32;&#35;18&#46;76fed417&#46;1741702909&#46;9254f33
<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;76fed417&#46;1741702909&#46;9254f33</P>
</BODY>
</HTML>



In [27]:
response.text

'<HTML><HEAD>\n<TITLE>Access Denied</TITLE>\n</HEAD><BODY>\n<H1>Access Denied</H1>\n \nYou don\'t have permission to access "http&#58;&#47;&#47;www&#46;meesho&#46;com&#47;men&#45;watches&#47;pl&#47;3k7" on this server.<P>\nReference&#32;&#35;18&#46;25fed417&#46;1741702811&#46;737f87\n<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;25fed417&#46;1741702811&#46;737f87</P>\n</BODY>\n</HTML>\n'

In [23]:
webpage = response.text
soup = BeautifulSoup(webpage,"lxml")

In [25]:
print(soup.prettify())

<html>
 <head>
  <title>
   503 - Service Unavailable Error
  </title>
 </head>
 <body bgcolor="#FFFFFF" text="#000000">
  <!--
        To discuss automated access to Amazon data please contact api-services-support@amazon.com.
        For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.in/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.in/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.
-->
  <center>
   <a href="https://www.amazon.in/ref=cs_503_logo/">
    <img alt="Amazon.in" border="0" height="45" src="https://images-eu.ssl-images-amazon.com/images/G/31/x-locale/communities/people/logo.gif" width="200"/>
   </a>
   <p align="center">
    <font face="Verdana,Arial,Helvetica">
     <font color="#CC6600" size="+2">
      <b>
       Oops!
      </b>
     </font>
     <br/>
     <b>
      It's rush hour and traffic is piling up on that page. Please try again in a short whil

In [202]:
from selenium import webdriver
import selenium.webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random
import os
import numpy as np
import pandas as pd

In [72]:
os.chdir(r"D:\Jupyter\flipkart")
os.getcwd()

'D:\\Jupyter\\flipkart'

In [75]:
file_list = os.listdir()

In [73]:
s = Service("C:/Users/ASUS/Desktop/chromedriver.exe")
driver = webdriver.Chrome(service=s)

#all_data = []
for i in range(1,37):
    driver.get("https://www.flipkart.com/search?sid=tyy%2C4io&otracker=CLP_Filters&p%5B%5D=facets.brand%255B%255D%3DSAMSUNG&page={}".format(i))
    html = driver.page_source
    time.sleep(random.uniform(5, 15))
    with open(f"flipkart_data{i}.html","w",encoding="utf-8") as f:
        f.write(html)

In [76]:
from bs4 import BeautifulSoup

file_list
merged_soup = BeautifulSoup("<html><head></head><body></body></html>", "html.parser")

for file in file_list:
    with open(file, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
        if soup.body:
            merged_soup.body.extend(soup.body.contents)

output_file = "merged.html"
with open(output_file, "w", encoding="utf-8") as outfile:
    outfile.write(str(merged_soup.prettify()))

print("HTML files merged successfully inside a single <body>!")

HTML files merged successfully inside a single <body>!


In [24]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# import time

# s = Service("C:/Users/ASUS/Desktop/chromedriver.exe")
# driver = webdriver.Chrome(service=s)

# html = driver.page_source

# driver.get("https://www.flipkart.com/search?sid=tyy%2C4io&p%5B%5D=facets.brand%255B%255D%3DSAMSUNG&page=1")
# time.sleep(5)

# products = driver.find_elements(By.XPATH, '//div[contains(@class, "_4rR01T")]')

# for product in products:
#     print(product.text)  # Print product names

# driver.quit()

In [77]:
with open("merged.html",encoding='utf-8') as f:
    html = f.read()

In [78]:
soup = BeautifulSoup(html,"lxml")

In [79]:
container = soup.find_all("div",class_="tUxRFH")

In [276]:
model_name = []
photo_img_url = []
star_rating = []
rating = []
review = []
discount_price = []
original_price = []
ram = []
display = []
camera = []
battery = []
processor = []
warranty = []
prod = 1

for i in container:
    model_name.append(i.find("div",class_="KzDlHZ").text.strip())
    photo_img_url.append(i.find("img",class_="DByuf4")["src"])
    try:
        star_rating.append(i.find("div",class_="XQDdHH").text.strip())
    except:
        star_rating.append("0")
    
    data = i.find_all("span",class_="Wphh3N")
    if data == []:
        rating.append(np.nan)
        review.append(np.nan)
        
    else:
        for j in data:
            try:
                text = j.text.strip()  # Get text safely
                words = text.split()  # Split into words
                if len(words)>=1:
                    rating.append(words[0])
                else:
                    rating.append(np.nan)

                if len(words) >=4:
                    review.append(words[3])
                else:
                    review.append(np.nan)
            except Exception as e:
                print(f"Error: {e}")
                rating.append(np.nan)
                review.append(np.nan) 
            
    try:
        discount_price.append(i.find("div",class_="Nx9bqj _4b5DiR").text.strip())
    except:
        discount_price.append(np.nan)
    try:
        original_price.append(i.find("div",class_="yRaY8j ZYYwLA").text.strip())
    except:
        original_price.append(np.nan)

    li = i.find_all("li",class_="J+igdf")
    spec = []
    for i in li:
        spec.append(i.text.strip())
    ram.append(spec[0])
    display.append(spec[1])
    camera.append(spec[2])
    battery.append(spec[3])
    try:
        processor.append(spec[4])
    except:
        processor.append(np.nan)
    try:
        warranty.append(spec[5])
    except:
        warranty.append(np.nan)

In [277]:
samsung_df = pd.DataFrame({
    "name" : model_name,
    "photo_img_url" : photo_img_url,
    "star_rating" : star_rating,
    "rating" : rating,
    "review" : review,
    "discount_price" : discount_price,
    "original_price" : original_price,
    "ram" : ram,
    "display" : display,
    "camera" : camera,
    "battery" : battery,
    "processor":processor,
    "warranty" : warranty
})

In [282]:
samsung_df.to_csv("samsung.csv",encoding="utf-8")

## gitbub : https://github.com/Mahi1503/Webscrapping_Samsung_flipkart_data